In [7]:
import re
import torch
import numpy as np
import pandas as pd
from torch_geometric.data import Data

In [8]:
# read in txt file with extra info
# construct adjacency list
datapath = "scripts/parcel_neighbors.txt"
features = pd.read_csv(datapath, delimiter="\t", header=None)
features.columns = ['parcel', 'network', 'neighbors']

In [9]:
def interpret_neighbors(neighbors):
    temp = neighbors.split(" ")
    temp = [int(el) for el in temp if el != '']
    return temp

# formatting the list of neighboring regions
features['neighbors'] = features['neighbors'].apply(lambda x: interpret_neighbors(x))

In [10]:
# creating an edge list 
edge_col = []
for i,row in features.iterrows():
    neighbors = row['neighbors']
    edge_list = [[i+1, el] for el in neighbors]

    # converting to a tensor
    edge_tensor = torch.tensor(edge_list, dtype=torch.long).t().contiguous()
    edge_col.append(edge_tensor)
edge_col = pd.Series(edge_col)
features['edgeList_embeddings'] = edge_col
# features = pd.concat([features, edge_col], axis=1)

In [11]:
seven_networks = {
    'Vis':         0,
    'SomMot':      1,
    'DorsAttn':    2,
    'SalVentAttn': 3,
    'Limbic':      4,
    'Cont':        5,
    'Default':     6,

}

split_network_info = features['network'].apply(lambda x: x.split("_"))
network_embedding_col = []
for net in split_network_info:
    network_embedding = torch.zeros(8)
    net_idx = seven_networks[net[2]]
    network_embedding[net_idx] = 1
    if net[1] == 'LH':
        network_embedding[7] = 1
    network_embedding_col.append(network_embedding)
network_embedding_col = pd.Series(network_embedding_col)
features['network_embeddings'] = network_embedding_col
# features

In [12]:
features

,parcel,network,neighbors,edgeList_embeddings,network_embeddings
0,1,7Networks_LH_Vis_1,"[2, 3, 4, 7, 69, 123, 126]","[[tensor(1), tensor(1), tensor(1), tensor(1), ...","[tensor(1.), tensor(0.), tensor(0.), tensor(0...."
1,2,7Networks_LH_Vis_2,"[1, 4, 7, 69, 121, 123, 126]","[[tensor(2), tensor(2), tensor(2), tensor(2), ...","[tensor(1.), tensor(0.), tensor(0.), tensor(0...."
2,3,7Networks_LH_Vis_3,"[1, 4, 5, 6, 8, 69, 70]","[[tensor(3), tensor(3), tensor(3), tensor(3), ...","[tensor(1.), tensor(0.), tensor(0.), tensor(0...."
3,4,7Networks_LH_Vis_4,"[1, 2, 3, 5, 7, 10, 13, 16]","[[tensor(4), tensor(4), tensor(4), tensor(4), ...","[tensor(1.), tensor(0.), tensor(0.), tensor(0...."
4,5,7Networks_LH_Vis_5,"[3, 4, 6, 8, 9, 10, 11, 18]","[[tensor(5), tensor(5), tensor(5), tensor(5), ...","[tensor(1.), tensor(0.), tensor(0.), tensor(0...."
...,...,...,...,...,...
395,396,7Networks_RH_Default_pCunPCC_5,"[192, 194, 195, 199, 200, 224, 225, 228, 356, ...","[[tensor(396), tensor(396), tensor(396), tenso...","[tensor(0.), tensor(0.), tensor(0.), tensor(0...."
396,397,7Networks_RH_Default_pCunPCC_6,"[147, 192, 196, 197, 198, 200, 260, 313, 316, ...","[[tensor(397), tensor(397), tensor(397), tenso...","[tensor(0.), tensor(0.), tensor(0.), tensor(0...."
397,398,7Networks_RH_Default_pCunPCC_7,"[49, 108, 109, 146, 147, 197, 198, 250, 254, 2...","[[tensor(398), tensor(398), tensor(398), tenso...","[tensor(0.), tensor(0.), tensor(0.), tensor(0...."
398,399,7Networks_RH_Default_pCunPCC_8,"[195, 200, 356, 357, 395, 396, 397, 400]","[[tensor(399), tensor(399), tensor(399), tenso...","[tensor(0.), tensor(0.), tensor(0.), tensor(0...."


In [ ]:
test = features.iloc[0,:]
torch.cat([test['edgeList_embeddings'], test['network_embeddings']])

RuntimeError: Tensors must have same number of dimensions: got 2 and 1

In [19]:
test['edgeList_embeddings']

tensor([[  1,   1,   1,   1,   1,   1,   1],
        [  2,   3,   4,   7,  69, 123, 126]])